In [1]:
from pyquil import Program, get_qc
from pyquil.gates import *
from pyquil.api import WavefunctionSimulator

## Algoritmo de Deutsch-Jozsa para 3 qubits

In [4]:
n = 3

In [5]:
def circuitoIni(n):
    qc = Program()
    qc += X(qubit = n)
    for i in range(n+1):
        qc += H(qubit = i)
    return qc

In [7]:
qc = circuitoIni(n)
print (qc)

X 3
H 0
H 1
H 2
H 3



In [8]:
def circuitoFin(n):
    qc3 = Program()
    for i in range(n):
        qc3 += H(qubit = i)
    return qc3

In [9]:
qc = circuitoFin(n)
print (qc)

H 0
H 1
H 2



### Constantes para 3 qubits

In [10]:
constante0 = Program()
print(constante0)

In [11]:
constante1 = Program()
constante1 += CNOT(0,n)
constante1 += X(qubit = 0)
constante1 += CNOT(0,n)
constante1 += CNOT(1,n)
constante1 += X(qubit = 1)
constante1 += CNOT(1,n)
constante1 += CNOT(2,n)
constante1 += X(qubit = 2)
constante1 += CNOT(2,n)
print(constante1)


CNOT 0 3
X 0
CNOT 0 3
CNOT 1 3
X 1
CNOT 1 3
CNOT 2 3
X 2
CNOT 2 3



### Balanceado para 3 qubits

In [36]:
balanced1 = Program()

balanced1 += CNOT(0,n)
balanced1 += CNOT(1,n)
balanced1 += CNOT(n-1,n)

print(balanced1)

CNOT 0 3
CNOT 1 3
CNOT 2 3



In [37]:
balanced0 = Program()
balanced0 += CNOT(0,n)
print(balanced0)

CNOT 0 3



In [38]:
#anhadimos los oraculos en el diccionario
oraculos = dict()
oraculos['constante0'] = constante0
oraculos['constante1'] = constante1
oraculos['balanced1'] = balanced1
oraculos['balanced0'] = balanced0

In [39]:
def deutsch_jozsa(n, oraculo):
    qc = Program()
    qc += circuitoIni(n)
    qc +=oraculos[oraculo]
    qc +=circuitoFin(n)
    return qc

In [40]:
qc = get_qc('4q-qvm') # un computador cuantica de 2 qubits para poder hacer la simulacion.
qc.qubits()

[0, 1, 2, 3]

### Contante 0

In [41]:
constante0Circuit = deutsch_jozsa(n, 'constante0')
print(constante0Circuit)

X 3
H 0
H 1
H 2
H 3
H 0
H 1
H 2



In [42]:
wf_sim = WavefunctionSimulator()
wavefunction = wf_sim.wavefunction(constante0Circuit)

In [43]:
print(wavefunction)

(0.7071067812+0j)|0000> + (-0.7071067812+0j)|1000>


In [44]:
bitstrings = qc.run_and_measure(constante0Circuit, trials=10)
bitstrings

{0: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 1: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 1, 0, 0, 0, 1, 0, 0, 1, 1])}

Como vemos, al aplicar el algoritmo, el qubit 0,1,2 nos ha devuelto todos los estado a 0. En cambio, el qubit 3 tiene un 50% de ser 1 o 0. Por lo que nuestro algortimo con el oraculo funciona.

### Contante 1

In [45]:
constante1Circuit = deutsch_jozsa(n, 'constante1')
print(constante1Circuit)

X 3
H 0
H 1
H 2
H 3
CNOT 0 3
X 0
CNOT 0 3
CNOT 1 3
X 1
CNOT 1 3
CNOT 2 3
X 2
CNOT 2 3
H 0
H 1
H 2



In [46]:
bitstrings = qc.run_and_measure(constante1Circuit, trials=10)
bitstrings

{0: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 1: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([0, 1, 0, 0, 1, 1, 0, 0, 0, 0])}

Como vemos, con el oraculo constante, nos dio el resultado de esperado, un probabilidad de 100% en los bits 000.

### Balanceado 0

In [47]:
balanced0Circuit = deutsch_jozsa(n, 'balanced0')
print(balanced0Circuit)

X 3
H 0
H 1
H 2
H 3
CNOT 0 3
H 0
H 1
H 2



In [48]:
bitstrings = qc.run_and_measure(balanced0Circuit, trials=10)
bitstrings

{0: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 1: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 2: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 3: array([1, 1, 0, 0, 0, 1, 0, 1, 0, 1])}

Como vemos, al aplicar el algoritmo, los tres qubits menos significativos nos ha devuelto todos los estado distintos de 00. En cambio, el qubit 3 tiene un 50% de ser 1 o 0. Por lo que nuestro algortimo con el oraculo funciona.

### Balanceado 1

In [49]:
balanced1Circuit = deutsch_jozsa(n, 'balanced1')
print(balanced1Circuit)

X 3
H 0
H 1
H 2
H 3
CNOT 0 3
CNOT 1 3
CNOT 2 3
H 0
H 1
H 2



In [50]:
bitstrings = qc.run_and_measure(balanced1Circuit, trials=10)
bitstrings

{0: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 1: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 2: array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1]),
 3: array([0, 1, 1, 0, 0, 0, 0, 1, 0, 1])}

Como vemos, al aplicar el algoritmo, los tres qubits menos significativos nos ha devuelto todos los estado distintos de 000. En cambio, el qubit 3 tiene un 50% de ser 1 o 0. Por lo que nuestro algortimo con el oraculo funciona.